# imports

In [ ]:
%matplotlib inline

In [ ]:
from IPython.core import debugger as idb

In [ ]:
# export
import pandas as pd

In [ ]:
# export
import re

In [ ]:
# export
import numpy as np

In [ ]:
# export
from fastai.vision import *

In [ ]:
# export
import os

In [ ]:
# export
import cv2

In [ ]:
# export
from matplotlib import pyplot asr plt

In [ ]:
# export
import random

In [ ]:
# export
#from FLAI.detect_symbol.exp import databunch as databunch_detsym

# functions

In [ ]:
#export
pat_coord = re.compile(r'\d+')
#图片在data_root目录下面的image目录里面，数字命名。
pat_imgName = re.compile(r'(\w+/\d+\.jpg)$')
pat_clas = re.compile(r'\w+')
pat_num = re.compile(r'\d+')    

def get_label_from_df(fn, df, pat_imgName, coord_col, cat_col, asbbox = False):
    '''
    fn: 
        file path.
    df: 
        a dataframe stores all the label information, imageName shoud be as index.
    repat_imgName: 
        a regular expression pattern, used to find the imageName from fn, where imageName is stored in df 
    box_col:
        the column name of bounding boxs
    cat_col:
        the column name of categories
    '''
    #print('glfd:', type(fn), fn)    
    fn = pat_imgName.findall(str(fn))[0]
    coords = df.loc[fn,coord_col]
    coords = pat_num.findall(coords)
    coords = list(map(np.long, coords))
    coords = np.array(coords).reshape(-1, 2) * 1.0    
    coords = coords.tolist()
    
    if asbbox:#暂时还沿用之前的bbox的形式。
        #import pdb;pdb.set_trace()
        ncoords = []
        for c in coords:
            ncoords += [[c[0], c[1], c[0] + 1, c[1] + 1]]
        coords = ncoords
    
    cats = df.loc[fn,cat_col]
    cats = pat_clas.findall(cats)
        
    return (coords, cats)

In [ ]:
def dbg():    
    data = PointsItemList.from_folder('./ds_20200818/image').split_by_rand_pct(valid_pct = 0.2)
    func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName
                   , coord_col='coord', cat_col='clas')
    import pdb;pdb.set_trace();
    data = data.label_from_func(func=func)
    
    data = data.databunch(bs=8, device=device, collate_fn=bb_pad_collate)
    data = data.normalize(imagenet_stats)
    
    return data
data = dbg()
data.show_batch()

In [ ]:
data = PointsItemList.from_folder('./ds_20200818/image').split_by_rand_pct(valid_pct = 0.2)

func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName
                   , coord_col='coord', cat_col='clas', asbbox = True)
data = data.label_from_func(func=func)

In [ ]:
data

In [ ]:


data = data.databunch(bs=8, device=device, collate_fn=bb_pad_collate)
data = data.normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [ ]:
data.train_ds

In [ ]:
# export
#描述坐标点的东西。
def ObjectCategoryList_get(self, i):
    return ImagePoints.create(*_get_size(self.x,i), *self.items[i], classes=self.classes, pad_idx=self.pad_idx)
#ObjectCategoryList.get = ObjectCategoryList_get

In [ ]:
# export
# 这个函数是为了在其它模块的设计时快速构造databunch
def get_databunch(data_root='./ds_20200818'
        , csv_name='gends.csv', valid_pct=0.2, bs=64, device=torch.device('cpu'), cache=False):
    '''
    --------------------------------
    参数：
    -- data_root：数据集的总目录
    -- img_path: 图片目录
    -- csv_name：存放标注信息的csv文件名，其要符合“对csv的要求”
    -- valid_pct：随机分割训练/验证集，该参数指定验证集的比例
    -- bs：batch size
    -- device：在datalaoder迭代时，dataloader先将batch加载到该device，做batch transform，然后返回。
    -- cache：dataset是否将所有图片预缓存入内存
    --------------------------------
    返回值：
    -- 一个databunch对象
    --------------------------------
    对csv的要求：
    1，带index
    2，存放图片名的列名称为"image"
    3，存放位置信息信息的列名称为"coord"
    4，存放类别信息的列名称为"clas"
    --------------------------------
    '''
    data_root = Path(data_root)
    csv_name = Path('gends.csv')
    # 读入csv，稍作处理，方便get_label函数操作
    csv_path = data_root/csv_name
    df = pd.read_csv(csv_path,index_col=0)
    df = df.set_index('image')

    # ItemList
    data = ObjectItemList.from_csv(path=data_root, csv_name=csv_name
                                   , cols='image')    

    # split ItemList to get ItemLists
    data = data.split_by_rand_pct(valid_pct=valid_pct)

    # label ItemLists to get LabelLists
    func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName
                   , coord_col='coord', cat_col='clas')
    data = data.label_from_func(func=func)

    # add transforms
#     trn_tfms = [*zoom_crop(scale=(0.9,1.1),do_rand=True,p=1),
#                 rot90_affine(use_on_y=True)]
#     val_tfms = []
#    data = data.transform(tfms=[trn_tfms,val_tfms], tfm_y=True, remove_out=True)
    # create DataBunch from LabelLists
    data = data.databunch(bs=bs, device=device, collate_fn=bb_pad_collate, num_workers=0)

    # normalize
    data = data.normalize()
    
    # 缓存图片
    if cache:
        data.cache_ds_img()
        
    return data

# test

In [ ]:
# 做些设置
data_root = './ds_20200818/'
data_root = Path(data_root)

csv_name = 'gends.csv'
csv_path = data_root/csv_name

img_subpath = 'image'
img_path = data_root/img_subpath

bs = 64


device = 'cpu'
device = torch.device('cuda')

In [ ]:
# 读入csv，稍作处理，方便get_label函数操作
df = pd.read_csv(csv_path,index_col=0)
df = df.set_index('image')
df.head()

In [ ]:
# ItemList
data = ObjectItemList.from_csv(path=data_root, csv_name=csv_name, cols='image')

In [ ]:
# split ItemList to get ItemLists
data = data.split_by_rand_pct(valid_pct=0.2)

In [ ]:
# def dbg(data):
#     import pdb; pdb.set_trace()
#     pat_imgName = re.compile(r'(\w+\/\w+\.jpg)$')
#     func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName, pos_col='pos')
#     data = data.label_from_func(func=func)
# dbg(data)    
    

In [ ]:
# label ItemLists to get LabelLists
func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName, coord_col='coord', cat_col='clas')
data = data.label_from_func(func=func)

In [ ]:
data


In [ ]:
# # add transforms
# trn_tfms = [*zoom_crop(scale=(0.9,1.1),do_rand=True,p=1),
#             rot90_affine(use_on_y=True)]
# val_tfms = []

# data = data.transform(tfms=[trn_tfms,val_tfms], tfm_y=True, remove_out=True)

In [ ]:
# create DataBunch from LabelLists
data = data.databunch(bs=8, device=device, collate_fn=bb_pad_collate)

In [ ]:
# 归一化
# 当bs较大时，可以不必传入stats参数，这是normalize函数内部会取data的一个batch来统计stats，并以此近似整个数据集的stats
# 但是当bs较小时，这样做就很不准确，需要先统计整个数据集的stats，在这里作为参数传入
data = data.normalize()

In [ ]:
# 缓存图片
# data.cache_ds_img()

In [ ]:
data = get_databunch(bs = 16, device = device)

In [ ]:
data.show_batch(rows=3)

In [ ]:
tdl = data.train_dl

In [ ]:
tdldl = tdl.dl

In [ ]:
tdldl.sampler

In [ ]:
# 查看统计信息
#databunch_detsym.databunch_statistics(data);

In [ ]:
#lesson3-head-pose.ipynb
data = PointsItemList.from_folder('./ds_20200818/image').split_by_rand_pct(valid_pct = 0.2)
func = partial(get_label_from_df, df=df, pat_imgName=pat_imgName
                   , coord_col='coord', cat_col='clas')
data = data.label_from_func(func=func)

data = data.databunch(bs=8, device=device, collate_fn=bb_pad_collate)
data = data.normalize(imagenet_stats)

In [ ]:
data

# export

In [ ]:
!python ../notebook2script.py --fname 'databunch.ipynb' --outputDir '../exp/'